# 通过叠加分析计算流域平均

In [1]:
import xarray as xr
from hydro_gistools import mean

利用xarray读入grid数据

In [2]:
ds=xr.open_dataset('3B-HHR-E.MS.MRG.3IMERG.20220917-S000000-E002959.0000.V06C.HDF5.SUB.nc4')
ds

<xarray.Dataset>
Dimensions:                         (time: 1, lon: 630, lat: 510)
Coordinates:
  * time                            (time) datetime64[ns] 2022-09-17
  * lon                             (lon) float32 73.05 73.15 ... 135.9 135.9
  * lat                             (lat) float32 3.05 3.15 3.25 ... 53.85 53.95
Data variables:
    precipitationCal                (time, lon, lat) float32 ...
    precipitationUncal              (time, lon, lat) float32 ...
    randomError                     (time, lon, lat) float32 ...
    HQprecipitation                 (time, lon, lat) float32 ...
    HQprecipSource                  (time, lon, lat) float32 ...
    HQobservationTime               (time, lon, lat) timedelta64[ns] ...
    IRprecipitation                 (time, lon, lat) float32 ...
    IRkalmanFilterWeight            (time, lon, lat) float32 ...
    probabilityLiquidPrecipitation  (time, lon, lat) float32 ...
Attributes:
    CDI:                                    Climate Data Interface version 1....
    Conventions:                            CF-1.6
    Original_Producer_Metadata_FileHeader:  DOI=10.5067/GPM/IMERG/3B-HH-E/06;...
    Original_Producer_Metadata_FileInfo:    DataFormatVersion=6a;\nTKCodeBuil...
    Original_Producer_Metadata_GridHeader:  BinMethod=ARITHMETIC_MEAN;\nRegis...
    InputPointer:                           3B-HHR-E.MS.MRG.3IMERG.20220917-S...
    history_L34RS:                          'Created by L34RS v1.4.3 @ NASA G...
    CDO:                                    Climate Data Operators version 1....

定位到具体时间time=0，此处0为时间序列索引，定位后ds的Coordinates里只保留经纬度。

In [3]:
ds=ds.isel(time=0)
ds

<xarray.Dataset>
Dimensions:                         (lon: 630, lat: 510)
Coordinates:
    time                            datetime64[ns] 2022-09-17
  * lon                             (lon) float32 73.05 73.15 ... 135.9 135.9
  * lat                             (lat) float32 3.05 3.15 3.25 ... 53.85 53.95
Data variables:
    precipitationCal                (lon, lat) float32 ...
    precipitationUncal              (lon, lat) float32 ...
    randomError                     (lon, lat) float32 ...
    HQprecipitation                 (lon, lat) float32 ...
    HQprecipSource                  (lon, lat) float32 ...
    HQobservationTime               (lon, lat) timedelta64[ns] ...
    IRprecipitation                 (lon, lat) float32 ...
    IRkalmanFilterWeight            (lon, lat) float32 ...
    probabilityLiquidPrecipitation  (lon, lat) float32 ...
Attributes:
    CDI:                                    Climate Data Interface version 1....
    Conventions:                            CF-1.6
    Original_Producer_Metadata_FileHeader:  DOI=10.5067/GPM/IMERG/3B-HH-E/06;...
    Original_Producer_Metadata_FileInfo:    DataFormatVersion=6a;\nTKCodeBuil...
    Original_Producer_Metadata_GridHeader:  BinMethod=ARITHMETIC_MEAN;\nRegis...
    InputPointer:                           3B-HHR-E.MS.MRG.3IMERG.20220917-S...
    history_L34RS:                          'Created by L34RS v1.4.3 @ NASA G...
    CDO:                                    Climate Data Operators version 1....

gpm和gfs数据是全国范围的，为了节省计算时间，建议先切片到松辽流域

In [4]:
ds=ds.sel(
    lat=slice(38,54),      # 注意有的是从大到小有的是从小到大
    lon=slice(115,136)     # 此处lat和lon要看ds中Coordinates里的名称，如era5数据中是latitude和longitude
)
ds

<xarray.Dataset>
Dimensions:                         (lon: 210, lat: 160)
Coordinates:
    time                            datetime64[ns] 2022-09-17
  * lon                             (lon) float32 115.1 115.2 ... 135.9 135.9
  * lat                             (lat) float32 38.05 38.15 ... 53.85 53.95
Data variables:
    precipitationCal                (lon, lat) float32 ...
    precipitationUncal              (lon, lat) float32 ...
    randomError                     (lon, lat) float32 ...
    HQprecipitation                 (lon, lat) float32 ...
    HQprecipSource                  (lon, lat) float32 ...
    HQobservationTime               (lon, lat) timedelta64[ns] ...
    IRprecipitation                 (lon, lat) float32 ...
    IRkalmanFilterWeight            (lon, lat) float32 ...
    probabilityLiquidPrecipitation  (lon, lat) float32 ...
Attributes:
    CDI:                                    Climate Data Interface version 1....
    Conventions:                            CF-1.6
    Original_Producer_Metadata_FileHeader:  DOI=10.5067/GPM/IMERG/3B-HH-E/06;...
    Original_Producer_Metadata_FileInfo:    DataFormatVersion=6a;\nTKCodeBuil...
    Original_Producer_Metadata_GridHeader:  BinMethod=ARITHMETIC_MEAN;\nRegis...
    InputPointer:                           3B-HHR-E.MS.MRG.3IMERG.20220917-S...
    history_L34RS:                          'Created by L34RS v1.4.3 @ NASA G...
    CDO:                                    Climate Data Operators version 1....

读入流域矢量数据

In [5]:
import geopandas as gpd
watershed=gpd.read_file('watershed_364.json')

利用[mean_over_basin](https://wis-processor.readthedocs.io/en/latest/modules/mean/#wis_processor.mean.mean_over_basin)方法计算流域平均

In [6]:
%%time
mean_tp=mean.mean_over_basin(watershed,'SID',ds,'precipitationCal',lon='lon',lat='lat')
mean_tp

CPU times: user 1min 40s, sys: 5.47 s, total: 1min 46s
Wall time: 1min 46s


SID
10412278    0.000347
10412314    0.000000
10412324    0.000000
10412338    0.000000
10412358    0.000000
              ...   
21401479    0.000000
21401489    0.000000
21401637    0.000000
21401827    0.000000
21401930    0.000000
Length: 364, dtype: float64